In [16]:
import wiki_helper
import importlib
importlib.reload(wiki_helper)
from wiki_helper import *

In [17]:
alias_map=load_alias()


In [18]:
support=5000
freq1=load_freqset("1-freq-"+str(support)+"supp.csv")
freq2=load_freqset("2-freq-"+str(support)+"supp.csv")

In [19]:
def generate_rules(freq2:dict,freq1:dict,min_conf:float):
    ret={}
    for key,value in tqdm(freq2.items()):
        rule_sup=len(value)
        item1,item2=list(key)
        if item1[1]=='_' and item2[1]=='_':
            continue
        if item1[0]=='Q' and item2[0]=='Q':
            continue
        if item2[1]=='_':
            item1,item2=item2,item1
        # item1 is an action, item2 is an entity
        if not item1 in freq1:
            item1_key=frozenset([item1])
        else:
            item1_key=item1
        item1_sup=len(freq1[item1_key])
        conf1=rule_sup/item1_sup
     
        if(conf1>=min_conf):
            if item1 not in ret:
                ret[item1]=set()
            if item2.__class__==str:
                ret[item1].add(item2)
            else:
                for i in item2:
                    ret[item1].add(i)

    return ret

In [20]:
rules=generate_rules(freq2,freq1,0.5)

100%|██████████| 525/525 [00:00<00:00, 29428.40it/s]


In [21]:
dump_rules(rules,"2-rules")

In [22]:
with open("output/b2-5000.csv",'w') as f:
    meta_path=[]
    for key,value in rules.items():
        if key[0]=='t':
            continue
        
        tail_key='t'+key[1:]
        if not tail_key in rules:
            continue
        else:
            tail_entities=rules[tail_key]
        head_entities=value
        for head in head_entities:
            for tail in tail_entities:
                meta_path.append((head,key[2:],tail))
                # f.write(head+','+key[2:]+','+tail+'\n')
                if head in alias_map:
                    f.write(alias_map[head])
                else:
                    f.write(head)
                f.write(',')
                if key[2:] in alias_map:
                    f.write(alias_map[key[2:]])
                else:
                    f.write(key[2:])
                f.write(',')
                if tail in alias_map:
                    f.write(alias_map[tail])
                else:
                    f.write(tail)
                f.write('\n')
                